In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on
import numpy as np

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
file_path="input/"
file_names=["0version_Box_TPC_Au_Au_ZeroField_1mrad_aligned_10evt_9nSkip_75570-0_resid.root"]
file_names=['Au_Au_seeds_37thevt_66522-0_resid.root']

In [3]:
# Global mode: use radial triplets everywhere
TRIPLET_MODE = "radial"

def assert_radial():
    global TRIPLET_MODE
    if TRIPLET_MODE != "radial":
        print("Warning: Forcing TRIPLET_MODE='radial'")
        TRIPLET_MODE = "radial"

print("TRIPLET_MODE:", TRIPLET_MODE)


TRIPLET_MODE: radial


In [4]:
hists_read, hists_sim = [], []

# Open ROOT files and keep them open so TTrees remain accessible later
open_files = []  # keep TFile references alive
trees = {
    "cluster": [],
    "residual": [],
    "hit": [],
    "vertex": []
}

for iFile in range(len(file_names)):
    fpath = file_path + file_names[iFile]
    tfile = root.TFile.Open(fpath, "READ")
    if not tfile or tfile.IsZombie():
        print(f"Failed to open {fpath}")
        trees["cluster"].append(None)
        trees["residual"].append(None)
        trees["hit"].append(None)
        trees["vertex"].append(None)
        continue

    open_files.append(tfile)

    # Retrieve trees if available (names from your file structure)
    trees["cluster"].append(tfile.Get("clustertree"))
    trees["residual"].append(tfile.Get("residualtree"))
    trees["hit"].append(tfile.Get("hittree"))
    trees["vertex"].append(tfile.Get("vertextree"))

# Handy shorthand to the first file's trees
cluster_tree = trees["cluster"][0] if trees["cluster"] else None
residual_tree = trees["residual"][0] if trees["residual"] else None
hit_tree = trees["hit"][0] if trees["hit"] else None
vertex_tree = trees["vertex"][0] if trees["vertex"] else None

print(f"Loaded files: {len(open_files)}")
print("cluster_tree entries:", cluster_tree.GetEntries() if cluster_tree else 0)
print("residual_tree entries:", residual_tree.GetEntries() if residual_tree else 0)

Loaded files: 1
cluster_tree entries: 13109
residual_tree entries: 161


In [5]:
# Quick check: list a few branches from the trees so they are clearly accessible
if cluster_tree:
    cluster_branches = [cluster_tree.GetListOfBranches().At(i).GetName() for i in range(min(10, cluster_tree.GetListOfBranches().GetEntries()))]
    print("cluster_tree branches (first 10):", cluster_branches)
if residual_tree:
    residual_branches = [residual_tree.GetListOfBranches().At(i).GetName() for i in range(min(10, residual_tree.GetListOfBranches().GetEntries()))]
    print("residual_tree branches (first 10):", residual_branches)

cluster_tree branches (first 10): ['run', 'segment', 'event', 'gl1bco', 'lx', 'lz', 'gx', 'gy', 'gz', 'phi']
residual_tree branches (first 10): ['run', 'segment', 'event', 'mbdcharge', 'mbdzvtx', 'firedTriggers', 'gl1BunchCrossing', 'trackid', 'tpcid', 'silid']


In [6]:
hist3d_xyz = root.TH3F("hist3d_xyz", "3D Residuals;X Residual;Y Residual;Z Residual", 100, -60, 60, 100, -60, 60, 100, -100, 100)
if cluster_tree:
    for entry in range(cluster_tree.GetEntries()):
        cluster_tree.GetEntry(entry)
        x_resid = cluster_tree.gx
        y_resid = cluster_tree.gy
        z_resid = cluster_tree.gz
        if (x_resid**2 + y_resid**2)**0.5 > 12:  # only fill if within 100 units
            hist3d_xyz.Fill(x_resid, y_resid, z_resid)


In [7]:
#c1 = root.TCanvas("c1", "3D Residuals", 800, 600)
#hist3d_xyz.Draw("BOX2")
#c1.Draw()

In [8]:

# Build a spatial index (R-tree-like) over cluster hits and provide triplet finder
# This keeps data in memory for fast neighbor queries in later cells.

# 1) Collect cluster points into a NumPy array [N,3]
cluster_points = None
cluster_entry_index = None  # maps point index -> TTree entry index

if cluster_tree:
    xs, ys, zs = [], [], []
    entry_idx = []
    n_entries = int(cluster_tree.GetEntries())
    for i in range(n_entries):
        cluster_tree.GetEntry(i)
        # Using branch names present in your file: gx, gy, gz
        xs.append(float(cluster_tree.gx))
        ys.append(float(cluster_tree.gy))
        zs.append(float(cluster_tree.gz))
        entry_idx.append(i)
    cluster_points = np.column_stack([xs, ys, zs]).astype(np.float32)
    cluster_entry_index = np.array(entry_idx, dtype=np.int64)
    print(f"cluster_points shape: {cluster_points.shape}")
else:
    print("No cluster_tree available; spatial index not built.")

cluster_points shape: (13109, 3)


In [9]:
# Convert cluster points to cylindrical coordinates (r, phi, z)
# Build spatial index in this space for radial triplet search

cluster_points_cyl = None  # [N, 3] array in (r, phi, z) coordinates

if cluster_points is not None:
    # Extract x, y, z
    x = cluster_points[:, 0]
    y = cluster_points[:, 1]
    z = cluster_points[:, 2]
    
    # Convert to cylindrical
    r = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    
    cluster_points_cyl = np.column_stack([r, phi, z]).astype(np.float32)
    
    print(f"Converted to cylindrical coordinates: {cluster_points_cyl.shape}")
    print(f"  r range: [{np.min(r):.2f}, {np.max(r):.2f}]")
    print(f"  phi range: [{np.min(phi):.2f}, {np.max(phi):.2f}]")
    print(f"  z range: [{np.min(z):.2f}, {np.max(z):.2f}]")
else:
    print("No cluster_points available for cylindrical conversion")


Converted to cylindrical coordinates: (13109, 3)
  r range: [1.85, 84.04]
  phi range: [-3.14, 3.14]
  z range: [-105.04, 103.79]


In [10]:
# Build spatial index in (r, phi, z) space
# For radial searches, we'll sort by r and use binning in phi and z

spatial_index_cyl = None
_index_backend_cyl = None

if cluster_points_cyl is not None and len(cluster_points_cyl):
    try:
        # Build KDTree in cylindrical space
        from scipy.spatial import cKDTree as _KDTree
        spatial_index_cyl = _KDTree(cluster_points_cyl)
        _index_backend_cyl = "scipy.cKDTree"
    except Exception:
        try:
            from sklearn.neighbors import KDTree as _SKKDTree
            spatial_index_cyl = _SKKDTree(cluster_points_cyl)
            _index_backend_cyl = "sklearn.KDTree"
        except Exception:
            spatial_index_cyl = None
            _index_backend_cyl = "none"

print("Cylindrical spatial index backend:", _index_backend_cyl)


Cylindrical spatial index backend: scipy.cKDTree


In [11]:
# Build spatial index in (r, phi, z) space
# For radial searches, we'll sort by r and use binning in phi and z

spatial_index_cyl = None
_index_backend_cyl = None

if cluster_points_cyl is not None and len(cluster_points_cyl):
    try:
        # Build KDTree in cylindrical space
        from scipy.spatial import cKDTree as _KDTree
        spatial_index_cyl = _KDTree(cluster_points_cyl)
        _index_backend_cyl = "scipy.cKDTree"
    except Exception:
        try:
            from sklearn.neighbors import KDTree as _SKKDTree
            spatial_index_cyl = _SKKDTree(cluster_points_cyl)
            _index_backend_cyl = "sklearn.KDTree"
        except Exception:
            spatial_index_cyl = None
            _index_backend_cyl = "none"

print("Cylindrical spatial index backend:", _index_backend_cyl)


Cylindrical spatial index backend: scipy.cKDTree


In [12]:
def neighbors_radius(i: int, radius: float):
    if _index_backend_cyl == "scipy.cKDTree":
        return spatial_index_cyl.query_ball_point(cluster_points[i], r=radius)
    elif _index_backend_cyl == "sklearn.KDTree":
        return spatial_index_cyl.query_radius(cluster_points[i:i+1], r=radius)[0].tolist()
    else:
        return spatial_index_cyl.radius_neighbors(i, radius)


In [13]:
import numpy as np

# Safety check
if cluster_points is None:
    raise RuntimeError("cluster_points is None. Run the earlier cells that build cluster_points from cluster_tree first.")

def estimate_local_directions_radius(points, radius=5.0):
    """
    For each hit i, look at neighbors within 'radius' (using neighbors_radius)
    and compute an approximate local direction vector.

    points: np.ndarray [N,3]
    returns: directions [N,3] unit vectors
    """
    N = points.shape[0]
    directions = np.zeros_like(points, dtype=np.float32)

    for i in range(N):
        p = points[i]
        neigh = neighbors_radius(i, radius)
        # Make sure neighbour list is a Python list of ints
        neigh = [j for j in neigh if j != i]

        if len(neigh) == 0:
            # Fallback arbitrary direction if isolated
            directions[i] = np.array([0.0, 0.0, 1.0], dtype=np.float32)
            continue

        neigh_pts = points[neigh]  # [k,3]
        vecs = neigh_pts - p[None, :]  # [k,3]

        # Average vector as approximate tangent
        mean_vec = vecs.mean(axis=0)
        norm = np.linalg.norm(mean_vec)
        if norm > 0:
            directions[i] = (mean_vec / norm).astype(np.float32)
        else:
            directions[i] = np.array([0.0, 0.0, 1.0], dtype=np.float32)

    return directions

# Example: compute directions with a 5 cm neighborhood
directions = estimate_local_directions_radius(cluster_points, radius=5.0)
print("directions shape:", directions.shape)


directions shape: (13109, 3)


In [14]:
# cluster_points_cyl: shape [N, 3] in (r, phi, z)
assert cluster_points_cyl is not None
assert cluster_points_cyl.shape[1] == 3

# unpack cylindrical coordinates
r_cyl   = cluster_points_cyl[:, 0].astype(float)
phi_raw = cluster_points_cyl[:, 1].astype(float)
z_cyl   = cluster_points_cyl[:, 2].astype(float)

# wrap phi into [-pi, pi] to be safe
phi_cyl = np.arctan2(np.sin(phi_raw), np.cos(phi_raw))

# polar angle relative to beam axis
theta_cyl = z_cyl#np.arctan2(r_cyl, z_cyl)

print("N hits (cyl):", len(cluster_points_cyl))
print("r   range:",   r_cyl.min(),   r_cyl.max())
print("phi range:",   phi_cyl.min(), phi_cyl.max())
print("theta range:", theta_cyl.min(), theta_cyl.max())


N hits (cyl): 13109
r   range: 1.8520758152008057 84.0386962890625
phi range: -3.1413896083831787 3.141516923904419
theta range: -105.04330444335938 103.7937240600586


In [15]:

def angle_diff(a, b):
    """
    Minimal signed difference between angles a and b (radians),
    result in [-pi, pi]
    """
    d = a - b
    d = (d + np.pi) % (2*np.pi) - np.pi
    return d


In [16]:
def neighbors_radius_cyl(i: int, radius: float):
    """
    Neighbours of hit i in cylindrical space (r,phi,z),
    using spatial_index_cyl built on cluster_points_cyl.
    'radius' is in that 3D Euclidean metric over (r,phi,z).
    """
    assert spatial_index_cyl is not None, "spatial_index_cyl is None"

    if _index_backend_cyl == "scipy.cKDTree":
        return spatial_index_cyl.query_ball_point(cluster_points_cyl[i], r=radius)

    elif _index_backend_cyl == "sklearn.KDTree":
        # sklearn KDTree expects a 2D array
        return spatial_index_cyl.query_radius(cluster_points_cyl[i:i+1], r=radius)[0].tolist()

    else:
        # fallback: brute force (slow, but safe)
        diffs = cluster_points_cyl - cluster_points_cyl[i]
        d2 = np.sum(diffs**2, axis=1)
        return np.where(d2 <= radius * radius)[0].tolist()


In [17]:
# cylindrical: cluster_points_cyl[:, 0] = r, [:, 1] = phi, [:, 2] = z
r   = cluster_points_cyl[:, 0]
phi = cluster_points_cyl[:, 1]
z   = cluster_points_cyl[:, 2]

x = r * np.cos(phi)
y = r * np.sin(phi)

cluster_points_xyz = np.column_stack([x, y, z])


In [18]:
from scipy.spatial import cKDTree

_index_backend_xyz = "scipy.cKDTree"
spatial_index_xyz = cKDTree(cluster_points_xyz)


In [19]:
def neighbors_radius_cyl(i: int, radius: float):
    """
    Neighbours of hit i in *real 3D space* (x,y,z),
    using spatial_index_xyz built on cluster_points_xyz.
    'radius' is in the same units as r,z (e.g. cm).
    """
    assert spatial_index_xyz is not None, "spatial_index_xyz is None"

    if _index_backend_xyz == "scipy.cKDTree":
        return spatial_index_xyz.query_ball_point(cluster_points_xyz[i], r=radius)

    elif _index_backend_xyz == "sklearn.KDTree":
        return spatial_index_xyz.query_radius(cluster_points_xyz[i:i+1], r=radius)[0].tolist()

    else:
        diffs = cluster_points_xyz - cluster_points_xyz[i]
        d2 = np.sum(diffs**2, axis=1)
        return np.where(d2 <= radius * radius)[0].tolist()


In [20]:
# Make sure things exist
assert 'neighbors_radius_cyl' in globals(), "neighbors_radius_cyl not defined."
assert 'r_cyl' in globals() and 'phi_cyl' in globals() and 'theta_cyl' in globals(), \
    "r_cyl, phi_cyl, theta_cyl must be defined from cluster_points_cyl."

def grow_chain_from_seed(
    seed_idx,
    used_mask,
    search_radius=5.0,
    min_step_dr=0.2,
    max_step_dr=6.0,
    max_dphi_step=0.10,
    max_dtheta_step=0.10,
    max_delta_r=2.0,
    max_delta_dphi=0.03,
    max_delta_dtheta=0.03,
    max_chain_hits=260,
):
    """
    Build one chain starting at seed_idx, going inward in r_cyl,
    with smooth dphi, dtheta in cylindrical coordinates.

    Returns list of hit indices in order (outer -> inner).
    """

    N = len(cluster_points_cyl)

    chain = [seed_idx]
    used_mask[seed_idx] = True

    prev_dphi = None
    prev_dtheta = None
    prev_dr = None

    current_idx = seed_idx

    for step in range(max_chain_hits - 1):
        rc     = r_cyl[current_idx]
        phic   = phi_cyl[current_idx]
        thetac = theta_cyl[current_idx]

        # Neighbours in (r,phi,z) from the cylindrical KDTree
        neigh = neighbors_radius_cyl(current_idx, search_radius)
        neigh = [j for j in neigh if j != current_idx and not used_mask[j]]

        if not neigh:
            break

        best_score = None
        best_idx = None
        best_dphi = None
        best_dtheta = None
        best_dr = None

        for j in neigh:
            rj     = r_cyl[j]
            phij   = phi_cyl[j]
            thetaj = theta_cyl[j]

            dr = rj - rc           # inward = negative
            if dr >= -min_step_dr:   # not going inward enough
                continue
            if dr < -max_step_dr:    # jump too large
                continue

            keff = 1.0
            if abs(dr) > 0.5: keff = abs(dr)
            dphi   = angle_diff(phij, phic)/keff
            dtheta = thetaj - thetac

            # per-step cuts
            if abs(dphi) > max_dphi_step:
                continue
            if abs(dtheta) > max_dtheta_step:
                continue

            # smoothness vs previous step, allowing missed hits (m = 1..4)
            ok = True
            if (prev_dphi is not None) or (prev_dtheta is not None) or (prev_dr is not None):
                ok = False
                for m in (1, 2, 3, 4):
                    conds = []

                    # radial step ~ m * prev_dr  (tolerance scales with m)
                    if prev_dr is not None:
                        conds.append(abs(dr - m * prev_dr) <= m * max_delta_r)

                    # angular step in XY: use wrapping-safe diff for phi
                    if prev_dphi is not None:
                        conds.append(abs(angle_diff(dphi, m * prev_dphi)) <= m * max_delta_dphi)

                    # angular step in RZ (theta)
                    if prev_dtheta is not None:
                        conds.append(abs(dtheta - m * prev_dtheta) <= m * max_delta_dtheta)

                    # accept if all applicable constraints pass
                    if conds and all(conds):
                        ok = True
                        break
                    
                if not ok:
                    continue


            if ( ( rj*np.cos(phij) - rc*np.cos(phic) )**2 +
                 ( rj*np.sin(phij) - rc*np.sin(phic) )**2 )**0.5 > search_radius:
                print("Warning: spatial_index_cyl inconsistent with real 3D distance!")
            # score: prefer smoother evolution
            if prev_dr is None:
                score = abs(dphi - (prev_dphi or 0.0)) + abs(dtheta - (prev_dtheta or 0.0))
            else:
                score = (
                    abs(dphi - (prev_dphi or 0.0))
                    + abs(dtheta - (prev_dtheta or 0.0))
                    + 0.2 * abs(dr - prev_dr)
                )

            if (best_score is None) or (score < best_score):
                best_score = score
                best_idx = j
                best_dphi = dphi
                best_dtheta = dtheta
                best_dr = dr

        if best_idx is None:
            break

        # accept continuation

        chain.append(best_idx)
        used_mask[best_idx] = True

        prev_dphi = best_dphi
        prev_dtheta = best_dtheta
        prev_dr = best_dr
        current_idx = best_idx

    return chain


In [21]:
def build_smooth_inward_chains(
    search_radius=5.0,
    min_step_dr=0.2,
    max_step_dr=6.0,
    max_dphi_step=0.10,
    max_dtheta_step=0.10,
    max_delta_dphi=0.03,
    max_delta_dtheta=0.03,
    min_chain_hits_keep=20,
    max_chain_hits_keep=48,
    max_delta_r=2.0
):
    """
    Loop over hits, sorted by r (outermost first),
    and grow inward chains that satisfy smoothness constraints.

    Returns: list of chains (each is list of hit indices).
    """
    N = len(cluster_points)
    # sort seeds outer->inner
    seed_order = np.argsort(-r_cyl)  # descending radius

    used_mask = np.zeros(N, dtype=bool)
    chains = []

    for seed_idx in seed_order:
        if used_mask[seed_idx]:
            continue

        # Optionally: skip very inner hits as seeds
        if r_cyl[seed_idx] < 20.0:   # e.g. don't seed from MVTX area
            continue

        chain = grow_chain_from_seed(
            seed_idx,
            used_mask,
            search_radius=search_radius,
            min_step_dr=min_step_dr,
            max_step_dr=max_step_dr,
            max_dphi_step=max_dphi_step,
            max_dtheta_step=max_dtheta_step,
            max_delta_dphi=max_delta_dphi,
            max_delta_dtheta=max_delta_dtheta,
            max_chain_hits=max_chain_hits_keep,
            max_delta_r=max_delta_r
        )

        if len(chain) >= min_chain_hits_keep and len(chain) <= max_chain_hits_keep:
            chains.append(chain)

    print(f"Built {len(chains)} smooth inward chains in [{min_chain_hits_keep},{max_chain_hits_keep}] hits.")
    return chains

# Run it
chains = build_smooth_inward_chains(
    search_radius=8.0,
    min_step_dr=-4.0,
    max_step_dr=4.0,
    max_dphi_step=0.2,
    max_dtheta_step=6,
    max_delta_dphi=0.05,
    max_delta_dtheta=2,
    min_chain_hits_keep=10,
    max_chain_hits_keep=248,
    max_delta_r=3.0
)

# Show lengths summary
lengths = [len(c) for c in chains]
if lengths:
    print("Chain length min / mean / max:",
          min(lengths), np.mean(lengths), max(lengths))
else:
    print("No chains found in the given range.")


Built 397 smooth inward chains in [10,248] hits.
Chain length min / mean / max: 10 19.09823677581864 123


In [22]:
# Create 3D ROOT plot of chains
if chains:
    c3d = root.TCanvas("c3d", "3D Chain Visualization", 1200, 900)
    
    # Create 3D histogram for the hit space
    x_min, x_max = cluster_points[:, 0].min(), cluster_points[:, 0].max()
    y_min, y_max = cluster_points[:, 1].min(), cluster_points[:, 1].max()
    z_min, z_max = cluster_points[:, 2].min(), cluster_points[:, 2].max()
    
    h3d_frame = root.TH3F("h3d_frame", "3D Chains;X [cm];Y [cm];Z [cm]",
                          1, x_min-10, x_max+10,
                          1, y_min-10, y_max+10,
                          1, z_min-10, z_max+10)
    h3d_frame.SetStats(0)
    h3d_frame.Draw()
    
    # Draw all hits as small markers (gray background)
    graph_all_hits = root.TGraph2D(len(cluster_points))
    for i in range(len(cluster_points)):
        graph_all_hits.SetPoint(i, 
                                cluster_points[i, 0], 
                                cluster_points[i, 1], 
                                cluster_points[i, 2])
    graph_all_hits.SetMarkerStyle(7)  # small dots
    graph_all_hits.SetMarkerColor(root.kGray)
    graph_all_hits.Draw("P0 SAME")
    
    # Draw chains as colored polylines
    polylines = []
    colors = [root.kRed, root.kRed+3, root.kGreen+2, root.kMagenta, root.kOrange, 
              root.kCyan, root.kViolet, root.kSpring, root.kTeal, root.kPink]
    
    max_chains_to_draw = min(1500, len(chains))
    for i in range(max_chains_to_draw):
        chain = chains[i]
    
        # --- polyline in chain order ---
        pl = root.TPolyLine3D(len(chain))
        for j, idx in enumerate(chain):
            x, y, z = cluster_points[idx]
            pl.SetPoint(j, float(x), float(y), float(z))
    
        pl.SetLineColor(colors[i % len(colors)])
        pl.SetLineWidth(4)
        pl.Draw()
        polylines.append(pl)
    
        # --- markers in chain order ---
        graph_chain = root.TGraph2D(len(chain))
        for j, idx in enumerate(chain):
            x, y, z = cluster_points[idx]
            graph_chain.SetPoint(j, float(x), float(y), float(z))
    
        graph_chain.SetMarkerStyle(20)
        graph_chain.SetMarkerSize(0.8)
        graph_chain.SetMarkerColor(colors[i % len(colors)])
        graph_chain.Draw("P0 SAME")
        polylines.append(graph_chain)

    
    c3d.Draw()
    
    print(f"3D ROOT plot: Showing {max_chains_to_draw} chains out of {len(chains)} total")
    print(f"Total hits displayed: {len(cluster_points)}")
else:
    print("No chains to plot")


3D ROOT plot: Showing 397 chains out of 397 total
Total hits displayed: 13109


In [23]:
outf = root.TFile("output/chains_display_peripheral.root", "RECREATE")
c3d.Write("c3d")   # serializes the canvas + its primitives
outf.Close()

In [24]:
do_continue = True

In [25]:
rack_cluster_points = None
track_cluster_entry_index = None  # maps point index -> TTree entry (track) index
N_of_tracks_standard = 0
N_clusters_ontrack_standard = 0
if residual_tree:
    txs, tys, tzs = [], [], []
    t_entry_idx = []
    n_entries = int(residual_tree.GetEntries())
    print(f"Building track cluster index from {n_entries} residual_tree entries...")
    for i in range(n_entries):
        residual_tree.GetEntry(i)

        nclus = len(residual_tree.clusgx)
       
        # Sanity check (optional)
        # assert nclus == len(residual_tree.clusgy) == len(residual_tree.clusgz)
        N_of_tracks_standard+=1
        for j in range(nclus):
            # collect positions

            #N_clusters_ontrack_standard+=1
            txs.append(float(residual_tree.clusgx[j]))
            tys.append(float(residual_tree.clusgy[j]))
            tzs.append(float(residual_tree.clusgz[j]))
            rtclus = (residual_tree.clusgx[j]**2 + residual_tree.clusgy[j]**2)**0.5
            if rtclus >20 and rtclus<80:
                N_clusters_ontrack_standard +=1

            # remember which track (tree entry) this cluster belongs to
            t_entry_idx.append(i)

    # shape: [N_total_clusters, 3]
    track_cluster_points = np.column_stack([txs, tys, tzs]).astype(np.float32)
    # shape: [N_total_clusters], values are tree entry indices (track ids)
    track_cluster_entry_index = np.array(t_entry_idx, dtype=np.int64)
    print(f"Number of tracks {N_of_tracks_standard}")
    print(f"Number of clusters on tracks {N_clusters_ontrack_standard}")
    print(f"track_cluster_points shape: {track_cluster_points.shape}")
    print(f"track_cluster_entry_index shape: {track_cluster_entry_index.shape}")
else:
    print("No residual_tree available; spatial index not built.")

Building track cluster index from 161 residual_tree entries...
Number of tracks 161
Number of clusters on tracks 5189
track_cluster_points shape: (5579, 3)
track_cluster_entry_index shape: (5579,)


In [26]:
#removing clusters that are in track_cluster_points but not in cluster_points with some tolerance
if cluster_points is not None and track_cluster_points is not None: 
    print("Before cleaning: cluster_points:", cluster_points.shape, " track_cluster_points:", track_cluster_points.shape)
    from scipy.spatial import cKDTree
    tree = cKDTree(cluster_points)
    indices_to_keep = []
    tolerance = 1  # adjust as needed
    for i, pt in enumerate(track_cluster_points):
        dist, idx = tree.query(pt, distance_upper_bound=tolerance)
        if dist != float('inf'):
            indices_to_keep.append(i)
    track_cluster_points = track_cluster_points[indices_to_keep]
    track_cluster_entry_index = track_cluster_entry_index[indices_to_keep]
    print("After cleaning: cluster_points:", cluster_points.shape, " track_cluster_points:", track_cluster_points.shape)


Before cleaning: cluster_points: (13109, 3)  track_cluster_points: (5579, 3)
After cleaning: cluster_points: (13109, 3)  track_cluster_points: (3323, 3)


In [27]:
#cleaning vtx hits from track_cluster_points so, all cluster with r< 25 cm are removed
if track_cluster_points is not None:
    print("Before cleaning vtx hits: track_cluster_points:", track_cluster_points.shape)
    r_track_clusters = np.sqrt(track_cluster_points[:,0]**2 + track_cluster_points[:,1]**2)
    vtx_mask = r_track_clusters >= 25.0
    removed_vtx = (~vtx_mask).sum()
    if removed_vtx:
        print(f"Removing {removed_vtx} track cluster points with r < 25 cm (vtx hits).")
        track_cluster_points = track_cluster_points[vtx_mask]
        track_cluster_entry_index = track_cluster_entry_index[vtx_mask]
    print("After cleaning vtx hits: track_cluster_points:", track_cluster_points.shape)

Before cleaning vtx hits: track_cluster_points: (3323, 3)
Removing 381 track cluster points with r < 25 cm (vtx hits).
After cleaning vtx hits: track_cluster_points: (2942, 3)


In [28]:
c_clust_standard = root.TCanvas("c_clust_standard", "All clusters and found on tracks", 1200, 900)

# Frame
x_min, x_max = cluster_points[:, 0].min(), cluster_points[:, 0].max()
y_min, y_max = cluster_points[:, 1].min(), cluster_points[:, 1].max()
z_min, z_max = -100, 100

h3d_frame = root.TH3F("h3d_frame", "3D Chains;X [cm];Y [cm];Z [cm]",
                        1, x_min-10, x_max+10,
                        1, y_min-10, y_max+10,
                        1, z_min-10, z_max+10)
h3d_frame.SetStats(0)
h3d_frame.Draw()

g_track_clusters = root.TGraph2D(len(track_cluster_points))
for i, (x, y, z) in enumerate(track_cluster_points):
    g_track_clusters.SetPoint(i, float(x), float(y), float(z))
g_track_clusters.SetMarkerStyle(20)
g_track_clusters.SetMarkerSize(0.6)
g_track_clusters.SetMarkerColor(root.kRed)
g_track_clusters.Draw("P0 SAME")


g_all_clusters = root.TGraph2D(len(cluster_points))
for i, (x, y, z) in enumerate(cluster_points):
    g_all_clusters.SetPoint(i, float(x), float(y), float(z))
g_all_clusters.SetMarkerStyle(7)
g_all_clusters.SetMarkerSize(0.1)
g_all_clusters.SetMarkerColor(root.kGray+1)
#g_all_clusters.Draw("P0 SAME")
for i in range(max_chains_to_draw):
    chain = chains[i]

    # --- polyline in chain order ---
    pl = root.TPolyLine3D(len(chain))
    for j, idx in enumerate(chain):
        x, y, z = cluster_points[idx]
        pl.SetPoint(j, float(x), float(y), float(z))

    pl.SetLineColor(colors[i % len(colors)])
    pl.SetLineWidth(4)
    pl.Draw()
    polylines.append(pl)

    # --- markers in chain order ---
    graph_chain = root.TGraph2D(len(chain))
    for j, idx in enumerate(chain):
        x, y, z = cluster_points[idx]
        graph_chain.SetPoint(j, float(x), float(y), float(z))

    graph_chain.SetMarkerStyle(20)
    graph_chain.SetMarkerSize(0.8)
    graph_chain.SetMarkerColor(colors[i % len(colors)])
    #graph_chain.Draw("P0 SAME")
    polylines.append(graph_chain)


# keep Python references so ROOT objects are not deleted
c_clust_standard._objs = [h3d_frame,  g_track_clusters, g_all_clusters] 
if do_continue: c_clust_standard.Draw()

In [29]:
print(f"Total number of clusters {len(cluster_points)}.")
N_clusters_chain = sum([len(c) for c in chains])
print(" ")
print(f"Number of clusters found on  tracks by standard algorithm {len(track_cluster_points)}.")
print(f"Number of clusters found on  chains {N_clusters_chain}.")
print(" ")
print(f"Number of  tracks by standard algorithm {N_of_tracks_standard}.")
print(f"Number of chains found {len(chains)}.")

Total number of clusters 13109.
 
Number of clusters found on  tracks by standard algorithm 2942.
Number of clusters found on  chains 7582.
 
Number of  tracks by standard algorithm 161.
Number of chains found 397.
